# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [6]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: Automate content creation, such as writing articles, social media posts, and product descriptions, to improve efficiency and consistency.
2. **Image and Video Creation**: Use generative models to create high-quality images and videos for marketing campaigns, advertising, and e-commerce websites.
3. **Product Design**: Utilize generative AI to design new products, such as furniture, clothing, or electronics, reducing the need for manual prototyping and improving product development speed.
4. **Predictive Maintenance**: Apply machine learning algorithms to predict equipment failures and schedule maintenance, reducing downtime and increasing overall efficiency.
5. **Customer Service Chatbots**: Develop conversational AI models that can understand customer queries and provide personalized responses, enhancing customer experience and support.
6. **Marketing Campaign Optimizatio

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [7]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can be used to create personalized content such as articles, social media posts, and even entire books. This technology can help reduce content creation costs and increase efficiency.
2. **Product Design**: Generative AI-powered design tools can generate 3D models, prototypes, and product designs, reducing the time and effort required for traditional design processes.
3. **Marketing Automation**: Generative AI can be used to create personalized marketing campaigns, automate lead generation, and even generate social media content.
4. **Customer Service**: Chatbots powered by Generative AI can provide 24/7 customer support, helping businesses improve response times and reduce costs.
5. **Personalized Recommendations**: Generative AI can be used to analyze customer behavior and preferences, providing personalized product recommendations in e-commerce, retail, and 

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI

In [20]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [21]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [22]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [23]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [24]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [28]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages)
    return response.message.content

In [26]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [29]:
display_summary("https://edwarddonner.com")

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can generate high-quality content such as blog posts, social media posts, and product descriptions, reducing the need for human writers.
2. **Marketing Automation**: AI-generated content can be used to personalize marketing messages, automate email campaigns, and optimize ad targeting for better ROI.
3. **Product Design**: Generative AI can create 3D models, product designs, and prototypes, streamlining the design process and reducing costs.
4. **Image and Video Generation**: AI-generated images and videos can be used for:
 * Social media content creation
 * Advertising and marketing campaigns
 * Product visualizations (e.g., e-commerce product pages)
5. **Music and Audio Generation**: Generative AI can create music, sound effects, and voiceovers for various applications, such as video games, film, and advertising.
6. **Chatbots and Virtual Assistants**: AI-powered chatbots can engage customers, provide support, and answer frequently asked questions, improving customer experience and reducing support costs.
7. **Predictive Maintenance**: Generative AI can analyze sensor data from machines to predict maintenance needs, reducing downtime and increasing overall equipment effectiveness (OEE).
8. **Risk Management**: AI-generated models can identify potential risks and threats in various industries, such as finance, healthcare, and energy, enabling proactive risk mitigation strategies.
9. **Data Augmentation**: Generative AI can augment existing datasets with new, synthetic data, improving model performance and reducing the need for large datasets.
10. **Customer Insights**: AI-powered analytics can generate insights on customer behavior, preferences, and interests, helping businesses tailor their offerings and marketing strategies.

Industry-specific applications:

1. **Finance**: AI-generated models can predict credit risk, detect fraud, and optimize investment portfolios.
2. **Healthcare**: Generative AI can analyze medical images, diagnose diseases, and develop personalized treatment plans.
3. **Retail**: AI-powered chatbots can engage customers, provide product recommendations, and improve the overall shopping experience.
4. **Manufacturing**: Generative AI can optimize production workflows, predict maintenance needs, and improve product quality.

These are just a few examples of the many business applications of Generative AI. As the technology continues to evolve, we can expect to see even more innovative use cases emerge across various industries.